In [2]:
!pip install psycopg2-binary

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 10.0 MB/s eta 0:00:00a 0:00:01


In [1]:
import pandas as pd
from configparser import ConfigParser
import psycopg2

In [2]:
def load_config(filename='database.ini', section='postgresql'):
    parser = ConfigParser()
    parser.read(filename)

    # get section, default to postgresql
    config = {}
    if parser.has_section(section):
        params = parser.items(section)
        for param in params:
            config[param[0]] = param[1]
    else:
        raise Exception('Section {0} not found in the {1} file'.format(section, filename))

    return config

def connect(config):
    """ Connect to the PostgreSQL database server """
    try:
        # connecting to the PostgreSQL server
        with psycopg2.connect(**config) as conn:
            print('Connected to the PostgreSQL server.')
            return conn
    except (psycopg2.DatabaseError, Exception) as error:
        print(error)

## Parsing data

In [3]:
df_train = pd.read_csv("../dataset/NYC/NYC_train.csv")
df_test = pd.read_csv("../dataset/NYC/NYC_test.csv")
df_val = pd.read_csv("../dataset/NYC/NYC_val.csv")

In [4]:
df_train = df_train[["POI_id", "POI_catname", "latitude", "longitude"]]
df_test = df_test[["POI_id", "POI_catname", "latitude", "longitude"]]
df_val = df_val[["POI_id", "POI_catname", "latitude", "longitude"]]

In [5]:
df_train

,POI_id,POI_catname,latitude,longitude
0,49bbd6c0f964a520f4531fe3,Arts & Crafts Store,40.719810,-74.002581
1,4a43c0aef964a520c6a61fe3,Bridge,40.606800,-74.044170
2,4c5cc7b485a1e21e00d35711,Home (private),40.716162,-73.883070
3,4ce1863bc4f6a35d8bd2db6c,Home (private),40.619151,-74.035888
4,4c0ab56f7e3fc9288c1df482,Mobile Phone Shop,40.741191,-73.989663
...,...,...,...,...
83223,50170a53e4b07ac9f310b944,Coffee Shop,40.709272,-74.005450
83224,4a9fc4fbf964a520493d20e3,Train Station,40.747582,-74.171888
83225,4d056fc57d9ba35d1c446b23,Building,40.760184,-73.969324
83226,4ae1f681f964a520318921e3,Clothing Store,40.753716,-73.980947


In [6]:
#Necessário apenas para o dataset da california
df = df_val

# Função para extrair o valor da chave 'name'
def extract_name(value):
    try:
        parsed_value = ast.literal_eval(value.replace(";", ","))  # Corrigindo delimitadores
        if isinstance(parsed_value, list) and len(parsed_value) > 0:
            return parsed_value[0].get("name", "")
    except (ValueError, SyntaxError):
        return ""
    
# Aplicando a transformação
df["POI_catname"] = df["POI_catname"].apply(extract_name)

# Exibindo o DataFrame transformado
print(df)

NameError: name 'ast' is not defined

In [6]:
df_all = pd.concat([df_train, df_test, df_val])
df_all

,POI_id,POI_catname,latitude,longitude
0,49bbd6c0f964a520f4531fe3,Arts & Crafts Store,40.719810,-74.002581
1,4a43c0aef964a520c6a61fe3,Bridge,40.606800,-74.044170
2,4c5cc7b485a1e21e00d35711,Home (private),40.716162,-73.883070
3,4ce1863bc4f6a35d8bd2db6c,Home (private),40.619151,-74.035888
4,4c0ab56f7e3fc9288c1df482,Mobile Phone Shop,40.741191,-73.989663
...,...,...,...,...
10334,4bec4056fd60a5932dba3af1,Office,40.910850,-74.031467
10335,4d2f32b573e5a0935c5158cf,Building,40.969114,-73.880592
10336,4d0304fc54d0236ac1a2e6d5,Hotel,40.965371,-74.062815
10337,4b857b2ff964a520b66031e3,City,40.817222,-74.003419


In [7]:
df_all.drop_duplicates(subset=["POI_id"], inplace=True)
df_all

,POI_id,POI_catname,latitude,longitude
0,49bbd6c0f964a520f4531fe3,Arts & Crafts Store,40.719810,-74.002581
1,4a43c0aef964a520c6a61fe3,Bridge,40.606800,-74.044170
2,4c5cc7b485a1e21e00d35711,Home (private),40.716162,-73.883070
3,4ce1863bc4f6a35d8bd2db6c,Home (private),40.619151,-74.035888
4,4c0ab56f7e3fc9288c1df482,Mobile Phone Shop,40.741191,-73.989663
...,...,...,...,...
5768,4e8e24c97ee6c06365762ab6,Bar,40.723503,-74.002845
8029,50c811dbe4b0d585092b704f,Other Great Outdoors,40.755905,-73.986869
8139,50c7104fe4b0860fb5620c72,Concert Hall,40.769634,-73.994346
8798,4d0382fa30a58cfa181399e7,Other Nightlife,40.754279,-73.987083


In [114]:
df_all.to_csv("../dataset/NYC/all_pois.csv")

In [5]:
df_all = pd.read_csv("../dataset/CA/all_pois.csv")
df_all.head()

,Unnamed: 0,POI_id,POI_catname,latitude,longitude
0,0,9174,Corporate Office,37.421059,-122.212952
1,1,9069,Corporate Office,37.444077,-122.163076
2,2,9175,Coffee Shop,37.444839,-122.161424
3,3,9668,University,37.723886,-122.479298
4,4,9583,Bar,37.783045,-122.465136


In [8]:
df_all.shape

(5099, 4)

## Adicionando no Banco

In [15]:
config = load_config()
conn = connect(config)
cursor = conn.cursor()

Connected to the PostgreSQL server.


In [10]:
cursor.execute("""CREATE TABLE poi (
               id VARCHAR(255) PRIMARY KEY,
               category VARCHAR(255),
               latitude FLOAT8,
               longitude FLOAT8);
""")

In [14]:
conn.commit()

cursor.close()
conn.close()

InterfaceError: connection already closed

In [16]:
for index, row in df_all.iterrows():
    cursor.execute(f"""INSERT INTO poi (id, category, latitude, longitude)
                   VALUES ('{row['POI_id']}', '{row['POI_catname']}', {row['latitude']}, {row['longitude']})

    """)

In [17]:
conn.commit()

cursor.close()
conn.close()